# Paquetes

In [2]:
import read_data as imp
import numpy as np
import pandas as pd
import pyfolio as pf
import matplotlib.pyplot as plt
import Alphas101 as Alphas

import matplotlib as plt
from math import sqrt
from yellowbrick.cluster import KElbowVisualizer
from statistics import stdev
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import kmeans,vq
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA 

'''Data Prep and Model Evaluation'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error
from keras.utils import to_categorical
from sklearn.metrics import adjusted_rand_score
import random

'''Algos'''
import tslearn
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import KShape, TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import dendrogram, linkage
from pyclustering.cluster.clarans import clarans;
from pyclustering.utils import timedcall;
from sklearn import datasets
import operator
import calendar
import itertools as it


import warnings
warnings.filterwarnings("ignore")

# Funciones Auxiliares

### Kshape

In [3]:
def strategy_kshape(df, n = 252, K = 4):
    X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df.T.values)

    ks = KShape(n_clusters=K, max_iter=100, n_init=100, verbose=0).fit(X)

    index = list(range(len(df.columns)))
    columns = ['strategies', 'clusters', 'selection']

    results = pd.DataFrame(index=index, columns=columns)

    results['clusters'] = ks.labels_
    results['strategies'] = df.columns

    sharpeclusters = []
    dfaux = df.tail(n)
    for i in range(results['clusters'].nunique()):
        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp = dfaux.pct_change().iloc[:, l].sum(axis=1, skipna=True)
        r = dfexp.mean()
        s = stdev(dfexp)

        sharpeclusters.append(r / s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

### Kmeans

In [4]:
def strategy_kmeans(df, n =2520, K =3):

    df = df.tail(n)
    returns = df.pct_change().mean()
    returns = pd.DataFrame(returns)
    returns.columns = ['Returns'] 
    returns['Volatility'] = df.pct_change().std()

    #format the data as a numpy array to feed into the K-Means algorithm
    data = np.asarray([np.asarray(returns['Returns']),np.asarray(returns['Volatility'])]).T
    
    n_samples = 1500
    random_state = 170

    # Number of clusters
    kmeans1 = KMeans(n_clusters=K, random_state=random_state)
    kmeans1.fit(data)
    #y_pred = kmeans1.fit_predict(X) 
    
    results=pd.DataFrame({'Returns': data[:, 0], 
                       'Volatility': data[:, 1],
                       'ClusterkMeans':kmeans1.labels_,
                       'strategies': df.columns,
                      })
    
    
    sharpeclusters =[] 
    for i in range(results['ClusterkMeans'].nunique()):
    
        l = results.loc[results['ClusterkMeans'] == i].index.values.astype(int).tolist()
        dfexp=df.pct_change().iloc[: , l].sum(axis = 1, skipna = True)
        r = dfexp.mean()
        s = stdev(dfexp)
    
        sharpeclusters.append(r/s)
    sharpeclusters = np.asanyarray(sharpeclusters)
    
    selection = sharpeclusters.argmax()
    cond = results['ClusterkMeans'] == selection
    results['selection'] = np.where(cond, 1, 0)
    
    
    return results['strategies'][results['selection'] == 1].values.tolist()
    
    

### TSKmeans - Euclidean

In [5]:
def strategy_tskmeans_euclidean (df, n =252, K =4, metric = "euclidean"):
    
    #  metric : {“euclidean”, “dtw”, “softdtw”} (default: “euclidean”)
    
    X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df.T.values)

    
    
    km = TimeSeriesKMeans(n_clusters=4, max_iter=200, n_init=200,tol = 1e-8, \
                      metric= metric, verbose=1, random_state=2019).fit(X)

    index = list(range(len(df.columns)))
    columns = ['strategies', 'clusters', 'selection']

    results = pd.DataFrame(index=index, columns=columns)

    results['clusters'] = km.labels_
    results['strategies'] = df.columns

    sharpeclusters = []
    dfaux = df.tail(n)
    for i in range(results['clusters'].nunique()):
        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp = dfaux.pct_change().iloc[:, l].sum(axis=1, skipna=True)
        r = dfexp.mean()
        s = stdev(dfexp)

        sharpeclusters.append(r / s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

### TSKmeans - dtw

In [6]:
def strategy_tskmeans_dtw (df, n =252, K =4, metric = "dtw"):
    
    #  metric : {“euclidean”, “dtw”, “softdtw”} (default: “euclidean”)
    
    X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df.T.values)

    
    
    km = TimeSeriesKMeans(n_clusters=4, max_iter=200, n_init=200,tol = 1e-8, \
                      metric= metric, verbose=1, random_state=2019).fit(X)

    index = list(range(len(df.columns)))
    columns = ['strategies', 'clusters', 'selection']

    results = pd.DataFrame(index=index, columns=columns)

    results['clusters'] = km.labels_
    results['strategies'] = df.columns

    sharpeclusters = []
    dfaux = df.tail(n)
    for i in range(results['clusters'].nunique()):
        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp = dfaux.pct_change().iloc[:, l].sum(axis=1, skipna=True)
        r = dfexp.mean()
        s = stdev(dfexp)

        sharpeclusters.append(r / s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

### TSKmeans - softdtw

In [7]:
def strategy_tskmeans_softdtw (df, n =252, K =4, metric = "softdtw"):
    
    #  metric : {“euclidean”, “dtw”, “softdtw”} (default: “euclidean”)
    
    X = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(df.T.values)

    
    
    km = TimeSeriesKMeans(n_clusters=4, max_iter=200, n_init=200,tol = 1e-8, \
                      metric= metric, verbose=1, random_state=2019).fit(X)

    index = list(range(len(df.columns)))
    columns = ['strategies', 'clusters', 'selection']

    results = pd.DataFrame(index=index, columns=columns)

    results['clusters'] = km.labels_
    results['strategies'] = df.columns

    sharpeclusters = []
    dfaux = df.tail(n)
    for i in range(results['clusters'].nunique()):
        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp = dfaux.pct_change().iloc[:, l].sum(axis=1, skipna=True)
        r = dfexp.mean()
        s = stdev(dfexp)

        sharpeclusters.append(r / s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

### Clarans

In [8]:
def clarans(df,k=3,v=4):
    df['year'] = df['Date'].apply(lambda x: x.split('-')[0])
    df2 = df[df['year'].isin(['2017','2018','2019'])]
    df2 = df2.drop('year',axis=1)
    df2=df2.set_index('Date').tail(365)
    df3 = df2.tail(365).transpose()
    data = np.array(df3)

    data = data.tolist()

    #get a glimpse of dataset
    #print("A peek into the dataset : ",data[:4])


    """!
    @brief Constructor of clustering algorithm CLARANS.
    @details The higher the value of maxneighbor, the closer is CLARANS to K-Medoids, and the longer is each search of a local minima.
    @param[in] data: Input data that is presented as list of points (objects), each point should be represented by list or tuple.
    @param[in] number_clusters: amount of clusters that should be allocated.
    @param[in] numlocal: the number of local minima obtained (amount of iterations for solving the problem).
    @param[in] maxneighbor: the maximum number of neighbors examined.        
    """
    clarans_instance = clarans(data, k, 6, v);

    #calls the clarans method 'process' to implement the algortihm
    (ticks, result) = timedcall(clarans_instance.process);
    print("Execution time : ", ticks, "\n");

    #returns the clusters 
    clusters = clarans_instance.get_clusters();

    #returns the mediods 
    medoids = clarans_instance.get_medoids();

    dic = {}
    for var in range(len(clusters)):
        for var2 in clusters[var]:
            dic[var2] = var


    resultado = sorted(dic.items(), key=operator.itemgetter(0))

    dic = {}
    for var in resultado:
        dic[df3.index[var[0]]] = var[1]

    clf = pd.DataFrame.from_dict(dic,orient='index')
    clf = clf.reset_index()
    results = clf.rename(columns={'index':'strategies',0:'clusters'})

    sharpeclusters =[] 
    Features = df2
    for i in range(results['clusters'].nunique()):

        l = results.loc[results['clusters'] == i].index.values.astype(int).tolist()
        dfexp=Features.pct_change().iloc[: , l].sum(axis = 1, skipna = True)
        r = dfexp.mean()
        s = dfexp.std()

        sharpeclusters.append(r/s)
    sharpeclusters = np.asanyarray(sharpeclusters)

    selection = sharpeclusters.argmax()
    cond = results['clusters'] == selection

    results['selection'] = np.where(cond, 1, 0)

    return results['strategies'][results['selection'] == 1].values.tolist()

In [22]:
def set_date(year_month,day):
    return str(year_month[0])+'-'+str(year_month[1])+'-'+str(day)

# Backtesting

In [20]:
Estrategias = pd.read_csv('Rentabilidad de estrategias base 100.csv')
Estrategias.set_index('Date', inplace = True)
Estrategias.index = pd.to_datetime(Estrategias.index)

1609

In [50]:
Initial_Date = pd.to_datetime('2014-01-01')

Rebalancing_dates = []
curr_month = Initial_Date.month

the_start = np.where(Initial_Date == Estrategias.index)[0].tolist()[0]

for i in range((the_start+1), len(Estrategias.index)):
    if Estrategias.index[i].month != curr_month:
        Rebalancing_dates.append(Estrategias.index[i-1])
        curr_month = Estrategias.index[i].month

## Calculating the factors over time

In [ ]:
for i in Strategy_Ret.index:
            
    if i in Rebalancing_dates:
        
        # update Baskets
        
        LA1_basket = pd.DataFrame({k:d['Alpha1'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA1_basket = list(LA1_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA1_basket = pd.DataFrame({k:d['Alpha1'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA1_basket = list(SA1_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA101_basket = pd.DataFrame({k:d['Alpha101'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA101_basket = list(LA101_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA101_basket = pd.DataFrame({k:d['Alpha101'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA101_basket = list(SA101_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA2_basket = pd.DataFrame({k:d['Alpha2'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA2_basket = list(LA2_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA2_basket = pd.DataFrame({k:d['Alpha2'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA2_basket = list(SA2_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA3_basket = pd.DataFrame({k:d['Alpha3'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA3_basket = list(LA3_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA3_basket = pd.DataFrame({k:d['Alpha3'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA3_basket = list(SA3_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA44_basket = pd.DataFrame({k:d['Alpha44'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA44_basket = list(LA44_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA44_basket = pd.DataFrame({k:d['Alpha44'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA44_basket = list(SA44_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA53_basket = pd.DataFrame({k:d['Alpha53'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA53_basket = list(LA53_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA53_basket = pd.DataFrame({k:d['Alpha53'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA53_basket = list(SA53_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA54_basket = pd.DataFrame({k:d['Alpha54'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA54_basket = list(LA54_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA54_basket = pd.DataFrame({k:d['Alpha54'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA54_basket = list(SA54_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        LA6_basket = pd.DataFrame({k:d['Alpha6'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LA6_basket = list(LA6_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SA6_basket = pd.DataFrame({k:d['Alpha6'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SA6_basket = list(SA6_basket.sort_values(by=['0']).index[0:Bkt_Size])
                
        LAMAR_basket = pd.DataFrame({k:d['AlphaMAR'][i] for k,d in hist_data.items()},index=['0']).transpose()
        LAMAR_basket = list(LAMAR_basket.sort_values(by=['0'],ascending=False).index[0:Bkt_Size])
        
        SAMAR_basket = pd.DataFrame({k:d['AlphaMAR'][i] for k,d in hist_data.items()},index=['0']).transpose()
        SAMAR_basket = list(SAMAR_basket.sort_values(by=['0']).index[0:Bkt_Size])
        
        # Return calculus
        
        Strategy_Ret.loc[i,['LA1']] = get_ret(hist_data, i, buy= LA1_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA1']] = get_ret(hist_data, i, sell= SA1_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA1']] = get_ret(hist_data, i, buy= LA1_basket, sell= SA1_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA101']] = get_ret(hist_data, i, buy= LA101_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA101']] = get_ret(hist_data, i, sell= SA101_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA101']] = get_ret(hist_data, i, buy= LA101_basket, sell= SA101_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA2']] = get_ret(hist_data, i, buy= LA2_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA2']] = get_ret(hist_data, i, sell= SA2_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA2']] = get_ret(hist_data, i, buy= LA2_basket, sell = SA2_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA3']] = get_ret(hist_data, i, buy= LA3_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA3']] = get_ret(hist_data, i, sell= SA3_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA3']] = get_ret(hist_data, i, buy= LA3_basket, sell= SA3_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA44']] = get_ret(hist_data, i, buy= LA44_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA44']] = get_ret(hist_data, i, sell= SA44_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA44']] = get_ret(hist_data, i, buy= LA44_basket, sell= SA44_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA53']] = get_ret(hist_data, i, buy= LA53_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA53']] = get_ret(hist_data, i, sell= SA53_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA53']] = get_ret(hist_data, i, buy= LA53_basket, sell= SA53_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA54']] = get_ret(hist_data, i, buy= LA54_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA54']] = get_ret(hist_data, i, sell= SA54_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA54']] = get_ret(hist_data, i, buy= LA54_basket, sell= SA54_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA6']] = get_ret(hist_data, i, buy= LA6_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA6']] = get_ret(hist_data, i, sell= SA6_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA6']] = get_ret(hist_data, i, buy= LA6_basket, sell= SA6_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LAMAR']] = get_ret(hist_data, i, buy= LAMAR_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SAMAR']] = get_ret(hist_data, i, sell= SAMAR_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSAMAR']] = get_ret(hist_data, i, buy= LAMAR_basket, sell= SAMAR_basket)*(1-com)*(1-BidAskSpread)
        
    else:
        
        Strategy_Ret.loc[i,['LA1']] = get_ret(hist_data, i, buy= LA1_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA1']] = get_ret(hist_data, i, sell= SA1_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA1']] = get_ret(hist_data, i, buy= LA1_basket, sell= SA1_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA101']] = get_ret(hist_data, i, buy= LA101_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA101']] = get_ret(hist_data, i, sell= SA101_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA101']] = get_ret(hist_data, i, buy= LA101_basket, sell= SA101_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA2']] = get_ret(hist_data, i, buy= LA2_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA2']] = get_ret(hist_data, i, sell= SA2_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA2']] = get_ret(hist_data, i, buy= LA2_basket, sell = SA2_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA3']] = get_ret(hist_data, i, buy= LA3_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA3']] = get_ret(hist_data, i, sell= SA3_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA3']] = get_ret(hist_data, i, buy= LA3_basket, sell= SA3_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA44']] = get_ret(hist_data, i, buy= LA44_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA44']] = get_ret(hist_data, i, sell= SA44_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA44']] = get_ret(hist_data, i, buy= LA44_basket, sell= SA44_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA53']] = get_ret(hist_data, i, buy= LA53_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA53']] = get_ret(hist_data, i, sell= SA53_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA53']] = get_ret(hist_data, i, buy= LA53_basket, sell = SA53_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA54']] = get_ret(hist_data, i, buy= LA54_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA54']] = get_ret(hist_data, i, sell= SA54_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA54']] = get_ret(hist_data, i, buy= LA54_basket, sell= SA54_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LA6']] = get_ret(hist_data, i, buy= LA6_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SA6']] = get_ret(hist_data, i, sell= SA6_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSA6']] = get_ret(hist_data, i, buy= LA6_basket, sell= SA6_basket)*(1-com)*(1-BidAskSpread)
        
        Strategy_Ret.loc[i,['LAMAR']] = get_ret(hist_data, i, buy= LAMAR_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['SAMAR']] = get_ret(hist_data, i, sell= SAMAR_basket)*(1-com)*(1-BidAskSpread)
        Strategy_Ret.loc[i,['LSAMAR']] = get_ret(hist_data, i, buy= LAMAR_basket, sell= SAMAR_basket)*(1-com)*(1-BidAskSpread)

In [ ]:
Strategy_Prices_TS = 100 * (Strategy_Ret+1).cumprod()
Strategy_Prices_TS.to_csv('Rentabilidad de estrategias base 100.csv')